# 구글 드라이브 마운트

In [1]:
from google.colab import drive

# 구글 드라이브와 연결
drive.mount('/content/drive')

mount_directory = "drive/MyDrive"

Mounted at /content/drive


# 기본 라이브러리 import

In [2]:
import pandas as pd
import numpy as np
import requests
import re
import os

from tqdm import tqdm

# 데이터셋 불러오기(MovieLens)

In [3]:
# 데이터 크기 설정 ex)latest-small, 32m, ...
dataset_volume = "32m"

# CSV 데이터 로드
df_ratings = pd.read_csv(f"{mount_directory}/ml-{dataset_volume}/ratings.csv")
df_movies = pd.read_csv(f"{mount_directory}/ml-{dataset_volume}/movies.csv")
df_tags = pd.read_csv(f"{mount_directory}/ml-{dataset_volume}/tags.csv")

In [4]:
# Dataset의 User, Movie 수 확인
n_users = df_ratings.userId.unique().shape[0]
n_items = df_ratings.movieId.unique().shape[0]
print("num users: {}, num items:{}".format(n_users, n_items))

num users: 200948, num items:84432


In [5]:
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama
87581,292737,Shelter in Solitude (2023),Comedy|Drama
87582,292753,Orca (2023),Drama
87583,292755,The Angry Breed (1968),Drama


# 데이터 전처리

In [6]:
# ratings의 timestamp 제거
df_ratings.drop(['timestamp'], axis=1, inplace=True)

In [7]:
# 데이터 전처리
# user id, movie id의 범위를 (0 ~ 사용자 수 -1), (0 ~ 영화 수 -1) 사이로 맞춰줌.

from scipy.sparse import coo_matrix

# user_id와 movie_id를 고유 인덱스로 매핑
user_dict = {user_id: idx for idx, user_id in enumerate(df_ratings['userId'].unique())}
movie_dict = {movie_id: idx for idx, movie_id in enumerate(df_ratings['movieId'].unique())}

# user_idx와 movie_idx로 새로운 컬럼 추가
df_ratings['user_idx'] = df_ratings['userId'].map(user_dict)
df_ratings['movie_idx'] = df_ratings['movieId'].map(movie_dict)

# 희소 행렬 생성
ratings_sparse = coo_matrix(
    (df_ratings['rating'], (df_ratings['user_idx'], df_ratings['movie_idx'])),
    shape=(len(user_dict), len(movie_dict))
)

# 역 매핑 딕셔너리 생성
user_idx_to_id = {v: k for k, v in user_dict.items()}
movie_idx_to_id = {v: k for k, v in movie_dict.items()}

In [8]:
# movie의 index로 부터 title과 genre 값으로 변환을 위한 dict 생성
movie_idx_to_name=dict()
movie_idx_to_genre=dict()
for row in df_movies.itertuples(index=False):
    movie_id, movie_name, movie_genre = row
    if movie_id not in movie_dict:              # 어떤 영화가 rating data에 없는 경우 skip
        continue
    movie_idx_to_name[movie_dict[movie_id]] = movie_name
    movie_idx_to_genre[movie_dict[movie_id]] = movie_genre

df_movies['genres'] = df_movies['genres'].apply(lambda x : x.split('|')).apply(lambda x : " ".join(x))

In [9]:
# 마지막 괄호 안의 연도를 추출하는 함수
def extract_year(title):
    match = re.search(r'\((\d{4})\)$', title)  # 문자열 끝에 있는 괄호 안의 숫자 찾기
    return int(match.group(1)) if match else None

In [10]:
# title에서 제목과 년도를 분리
df_movies['year'] = df_movies['title'].apply(extract_year)

# title에서 "( )"를 포함한 부연 설명 부분 제거
df_movies['title'] = df_movies['title'].str.split(" \(").str.get(0)

In [11]:
df_movies

,movieId,title,genres,year
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995.0
1,2,Jumanji,Adventure Children Fantasy,1995.0
2,3,Grumpier Old Men,Comedy Romance,1995.0
3,4,Waiting to Exhale,Comedy Drama Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0
...,...,...,...,...
87580,292731,The Monroy Affaire,Drama,2022.0
87581,292737,Shelter in Solitude,Comedy Drama,2023.0
87582,292753,Orca,Drama,2023.0
87583,292755,The Angry Breed,Drama,1968.0


# TMDB api를 통한 웹크롤링

감독, 배우 3명의 정보를 TMDB에 대한 api 요청을 통해 크롤링하여 Dataframe에 저장

이를 위해선 TMDB 홈페이지에 회원가입 및 api key를 할당 받아야 한다.

https://www.themoviedb.org/settings/api

In [12]:
# TMDb API 키 설정
API_KEY = "f2e3dc4096954042cfd1c618f01883ce"
BASE_URL = "https://api.themoviedb.org/3"

In [13]:
# 영화 제목으로 검색하여 감독과 배우 정보를 가져오는 함수
def get_movie_credits(title, year):
    # 1. 영화 ID 검색
    search_url = f"{BASE_URL}/search/movie"
    params = {"api_key": API_KEY, "query": title}
    if not np.isnan(year): # year 값이 없으면 title로만 요청
        params["year"] = int(year)

    response = requests.get(search_url, params=params)
    if response.status_code != 200 or not response.json()['results']:
        return None, None  # 데이터가 없으면 None 반환

    movie_id = response.json()['results'][0]['id']

    # 2. 영화 크레딧(감독/배우) 가져오기
    credits_url = f"{BASE_URL}/movie/{movie_id}/credits"
    credits_response = requests.get(credits_url, params={"api_key": API_KEY})
    if credits_response.status_code != 200:
        return None, None

    credits = credits_response.json()

    # 감독 정보 추출
    director = next((person['name'].replace(" ", "") for person in credits['crew'] if person['job'] == 'Director'), None)

    # 배우 정보 추출 (상위 3명)
    actors = " ".join(person['name'].replace(" ", "") for person in credits['cast'][:3])

    return director, actors

In [14]:
# 체크포인트 파일 경로 설정
save_folder = 'movies_processed_data'
save_file_name = f'movies_checkpoint.csv'

checkpoint_file = f"{mount_directory}/{save_folder}/{save_file_name}"

# 폴더 경로 생성
full_save_path = f"{mount_directory}/{save_folder}"

# 폴더가 없는 경우 생성
if not os.path.exists(full_save_path):
    os.makedirs(full_save_path)
    print(f"폴더를 생성했습니다: {full_save_path}")

# 체크포인트 데이터 로드
if os.path.exists(checkpoint_file):
    print(f"체크포인트 파일 로드 중: {checkpoint_file}")
    df_movies = pd.read_csv(checkpoint_file)
    print("체크포인트에서 이어서 작업을 진행합니다.")
else:
    print("새로운 작업을 시작합니다.")
    # df_movies 초기화: 원본 DataFrame을 로드하세요.
    # 예: df_movies = pd.read_csv('/path/to/original/movies.csv')

    # 'director'와 'actors' 열이 없으면 추가
    if 'director' not in df_movies.columns:
        df_movies['director'] = None
    if 'actors' not in df_movies.columns:
        df_movies['actors'] = None

        # 'check' 열 추가: 크롤링이 완료된 영화에는 'done' 값을 설정
    if 'check' not in df_movies.columns:
        df_movies['check'] = None

체크포인트 파일 로드 중: drive/MyDrive/movies_processed_data/movies_checkpoint.csv
체크포인트에서 이어서 작업을 진행합니다.


In [15]:
# 크롤링 작업
batch_size = 1000
total_batches = (len(df_movies) + batch_size - 1) // batch_size  # 정확히 1000개 단위로 분할

for batch_num in range(total_batches):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, len(df_movies))

    # 처리해야 할 데이터 필터링: 'check'가 None인 데이터만 처리
    batch_to_process = df_movies[start_idx:end_idx][df_movies['check'].isna()]

    if batch_to_process.empty:
        print(f"Batch {batch_num + 1}/{total_batches} 스킵 (이미 처리됨)")
        continue

    print(f"Batch {batch_num + 1}/{total_batches} 진행 중 ({start_idx} ~ {end_idx - 1})")

    for idx, row in tqdm(batch_to_process.iterrows(), total=batch_to_process.shape[0]):
        try:
            # 감독, 배우 정보 크롤링
            director, actor_list = get_movie_credits(row['title'], row['year'])
            df_movies.at[idx, 'director'] = director
            df_movies.at[idx, 'actors'] = actor_list
            df_movies.at[idx, 'check'] = 'done'  # 크롤링 완료된 항목은 'done'으로 표시
        except Exception as e:
            print(f"오류 발생 (index={idx}): {e}")
            df_movies.at[idx, 'check'] = 'error'  # 오류가 발생하면 'error'로 표시

    # 체크포인트 저장
    df_movies.to_csv(checkpoint_file, index=False, encoding='utf-8')
    print(f"Batch {batch_num + 1} 완료: 체크포인트 저장됨 ({checkpoint_file})")

print("모든 작업 완료!")

<ipython-input-15-20f28d74b8d5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  batch_to_process = df_movies[start_idx:end_idx][df_movies['check'].isna()]
<ipython-input-15-20f28d74b8d5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  batch_to_process = df_movies[start_idx:end_idx][df_movies['check'].isna()]
<ipython-input-15-20f28d74b8d5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  batch_to_process = df_movies[start_idx:end_idx][df_movies['check'].isna()]
<ipython-input-15-20f28d74b8d5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  batch_to_process = df_movies[start_idx:end_idx][df_movies['check'].isna()]
<ipython-input-15-20f28d74b8d5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  batch_to_process = df_movies[start_idx:end_idx][df_movies['check'].isna()]
<ipython-input-15-20f28d74b8d5>:10: UserWarning: Boolea

Batch 1/88 스킵 (이미 처리됨)
Batch 2/88 스킵 (이미 처리됨)
Batch 3/88 스킵 (이미 처리됨)
Batch 4/88 스킵 (이미 처리됨)
Batch 5/88 스킵 (이미 처리됨)
Batch 6/88 스킵 (이미 처리됨)
Batch 7/88 스킵 (이미 처리됨)
Batch 8/88 스킵 (이미 처리됨)
Batch 9/88 스킵 (이미 처리됨)
Batch 10/88 스킵 (이미 처리됨)
Batch 11/88 스킵 (이미 처리됨)
Batch 12/88 스킵 (이미 처리됨)
Batch 13/88 스킵 (이미 처리됨)
Batch 14/88 스킵 (이미 처리됨)
Batch 15/88 스킵 (이미 처리됨)
Batch 16/88 스킵 (이미 처리됨)
Batch 17/88 스킵 (이미 처리됨)
Batch 18/88 스킵 (이미 처리됨)
Batch 19/88 스킵 (이미 처리됨)
Batch 20/88 스킵 (이미 처리됨)
Batch 21/88 스킵 (이미 처리됨)
Batch 22/88 스킵 (이미 처리됨)
Batch 23/88 스킵 (이미 처리됨)
Batch 24/88 스킵 (이미 처리됨)
Batch 25/88 스킵 (이미 처리됨)
Batch 26/88 진행 중 (25000 ~ 25999)


100%|██████████| 1000/1000 [05:06<00:00,  3.26it/s]
<ipython-input-15-20f28d74b8d5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  batch_to_process = df_movies[start_idx:end_idx][df_movies['check'].isna()]


Batch 26 완료: 체크포인트 저장됨 (drive/MyDrive/movies_processed_data/movies_checkpoint.csv)
Batch 27/88 진행 중 (26000 ~ 26999)


100%|██████████| 1000/1000 [05:01<00:00,  3.32it/s]
<ipython-input-15-20f28d74b8d5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  batch_to_process = df_movies[start_idx:end_idx][df_movies['check'].isna()]


...

Batch 86 완료: 체크포인트 저장됨 (drive/MyDrive/movies_processed_data/movies_checkpoint.csv)
Batch 87/88 진행 중 (86000 ~ 86999)


100%|██████████| 1000/1000 [05:22<00:00,  3.10it/s]
<ipython-input-15-20f28d74b8d5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  batch_to_process = df_movies[start_idx:end_idx][df_movies['check'].isna()]


Batch 87 완료: 체크포인트 저장됨 (drive/MyDrive/movies_processed_data/movies_checkpoint.csv)
Batch 88/88 진행 중 (87000 ~ 87584)


100%|██████████| 585/585 [03:05<00:00,  3.16it/s]


Batch 88 완료: 체크포인트 저장됨 (drive/MyDrive/movies_processed_data/movies_checkpoint.csv)
모든 작업 완료!


In [18]:
df_movies.drop(['check'], axis=1, inplace=True)
df_movies

,movieId,title,genres,year,director,actors
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995.0,JohnLasseter,TomHanks TimAllen DonRickles
1,2,Jumanji,Adventure Children Fantasy,1995.0,JoeJohnston,RobinWilliams KirstenDunst BradleyPierce
2,3,Grumpier Old Men,Comedy Romance,1995.0,HowardDeutch,WalterMatthau JackLemmon Ann-Margret
3,4,Waiting to Exhale,Comedy Drama Romance,1995.0,ForestWhitaker,WhitneyHouston AngelaBassett LorettaDevine
4,5,Father of the Bride Part II,Comedy,1995.0,CharlesShyer,SteveMartin DianeKeaton MartinShort
...,...,...,...,...,...,...
87580,292731,The Monroy Affaire,Drama,2022.0,JosuéMéndez,DamiánAlcázar GrapaPaola MaríaZubiri
87581,292737,Shelter in Solitude,Comedy Drama,2023.0,VibekeMuasya,SiobhanFallonHogan PeterMacon RobertPatrick
87582,292753,Orca,Drama,2023.0,SaharMosayebi,TaranehAlidoosti MahtabKeramati MasoudKaramati
87583,292755,The Angry Breed,Drama,1968.0,DavidCommons,JanSterling JamesMacArthur WilliamWindom


# 크롤링한 데이터 csv 형식으로 Google Drive에 저장

In [19]:
# Google Drive 저장 경로 설정
save_folder = 'movies_processed_data'
save_file_name = f'movies_processed_{dataset_volume}.csv'

# 폴더 경로 생성
full_save_path = f"{mount_directory}/{save_folder}"

# 폴더가 없는 경우 생성
if not os.path.exists(full_save_path):
    os.makedirs(full_save_path)
    print(f"폴더를 생성했습니다: {full_save_path}")

# DataFrame을 CSV로 저장
df_movies.to_csv(f"{full_save_path}/{save_file_name}", index=False, encoding='utf-8')

print(f"CSV 파일이 저장되었습니다: {full_save_path}/{save_file_name}")

CSV 파일이 저장되었습니다: drive/MyDrive/movies_processed_data/movies_processed_32m.csv
